In [16]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [17]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle

    return angle 

In [25]:
#cap = cv2.VideoCapture("supino2-normal.mp4")

#cap = cv2.VideoCapture("puxada-frontal2.mp4")
cap = cv2.VideoCapture("puxada-frontal.mp4")
# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
           
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            print("Angulo: ", angle)
            if angle > 160:
                stage = "down"
            if angle < 80 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
cv2.destroyAllWindows()

Angulo:  123.44685260209982
Angulo:  78.54265614966118
Angulo:  74.91049584897075
Angulo:  106.00822209669761
Angulo:  160.9414437213195
Angulo:  72.66446605287433
1
Angulo:  101.04527305290912
Angulo:  80.12209949615317
Angulo:  80.40556019891893
Angulo:  91.94411085089814
Angulo:  95.49452033938542
Angulo:  88.7465290711105
Angulo:  80.2737705358465
Angulo:  91.81600786462138
Angulo:  97.62234847294354
Angulo:  89.00260246848941
Angulo:  87.80202926111697
Angulo:  77.39253933535544
Angulo:  75.83345767785625
Angulo:  70.50705977718222
Angulo:  77.8318342670575
Angulo:  83.62365793865665
Angulo:  89.70927104179978
Angulo:  49.506384416833384
Angulo:  59.27165976192957
Angulo:  67.74710371608418
Angulo:  78.31908717357636
Angulo:  86.76076649061116
Angulo:  100.9954905228442
Angulo:  87.73016530571569
Angulo:  78.45891647159019
Angulo:  73.15371444085207
Angulo:  72.32714097585033
Angulo:  69.33305182535172
Angulo:  71.92924687167833
Angulo:  71.39842574124059
Angulo:  87.3623912301927

In [24]:
cv2.destroyAllWindows()